In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import pyblock
from scipy.special import logsumexp
from scipy.special import softmax
import MDAnalysis as mda
from MDAnalysis.coordinates.PDB import PDBWriter
import os

# matplotlib settings
# --- Publication style settings ---
plt.rcParams.update({
    # Font
    "font.family": "Times New Roman",
    "font.serif": ["Arial"],  # or other preferred serif font
    "mathtext.fontset": "cm",            # Computer Modern for math text
    "text.usetex": False,                 # Use LaTeX for text rendering]
    "pdf.fonttype": 42,                 # Use LaTeX for text rendering

    # Figure
    "figure.figsize": (6, 4),            # width, height in inches
    "figure.dpi": 300,                   # high resolution

    # Axes
    "axes.titlesize": 14,
    "axes.labelsize": 12,
    "axes.linewidth": 1.2,
    "axes.grid": True,                   # optional, remove if you want no grid
    "grid.linestyle": "--",
    "grid.alpha": 0.7,

    # Ticks
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "xtick.direction": "in",
    "ytick.direction": "in",
    "xtick.top": True,
    "ytick.right": True,

    # Lines
    "lines.linewidth": 2,
    "lines.markersize": 6,

    # Legend
    "legend.fontsize": 10,
    "legend.frameon": False
})

# functions
# read_colvar function which sorts colvar for unique entries from the back (i.e., second entry retained)
def read_colvar(colvar_file):

    data = np.loadtxt(colvar_file)
    # inverse order
    data = data[::-1]
    unique_idxs = np.array(np.unique(data[:,0], return_index=True)[1]).astype(int)[::-1] # find unique indices, first occurences
    data = data[unique_idxs]
    # original order
    data = data[::-1]

    return data

# Function to calculate helical fraction
def helical_fraction(ss_line):
    helix_chars = {'H', 'G', 'I'}
    n_residues = len(ss_line)
    n_helical = sum(1 for c in ss_line if c in helix_chars)
    return n_helical / n_residues

# Function to calculate helical fraction
def betastrand_fraction(ss_line):
    beta_chars = {'E', 'B'}
    n_residues = len(ss_line)
    n_beta = sum(1 for c in ss_line if c in beta_chars)
    return n_beta / n_residues

# function to calculate PPII content
def PPII_fraction(ss_line):
    helix_chars = {'P'}
    n_residues = len(ss_line)
    n_helical = sum(1 for c in ss_line if c in helix_chars)
    return n_helical / n_residues

# Function to calculate per-residue helicity per frame
def dssp_to_helicity(dssp_lines):
    """
    Convert DSSP strings into a binary helicity matrix.
    
    Parameters
    ----------
    dssp_lines : list of str
        Each entry corresponds to one frame.
        Each string is the DSSP assignment per residue for that frame.
    
    Returns
    -------
    helicity : np.ndarray
        Array of shape (n_frames, n_residues).
        1 = helix ('H', 'G', 'I'), 0 = non-helix.
    """
    helix_codes = {'H', 'G', 'I'}
    
    n_frames = len(dssp_lines)
    n_residues = len(dssp_lines[0])
    
    helicity = np.zeros((n_frames, n_residues), dtype=int)
    
    for i, line in enumerate(dssp_lines):
        helicity[i, :] = [1 if c in helix_codes else 0 for c in line]
    
    return helicity





In [45]:
# AAQAA3
output_path = './pdbs_for_rendering/AAQAA3/'
path = '../OPES_multiT/AAQAA3/NEW_PRODUCTION_OPES_multiT_2fs_noHMR_300KrefT_48steps/alldata/'
data_path = './plots_AAQAA3_300K/'
sim = 0
n_structures = 10

def helical_fraction_H(ss_line):
    helix_chars = {'H'}
    n_residues = len(ss_line)
    n_helical = sum(1 for c in ss_line if c in helix_chars)
    return n_helical / n_residues

data = read_colvar(path+f'COLVAR.{sim}')
with open(path+f'dssp{sim}.dat') as f:
    dssp = [line.strip() for line in f]
# Time arrays in μs
time_data = data[:, 0] / 1e6        # ps -> μs
# Apply cutoff at 5 µs
mask_data = time_data <= 5.0 
data = data[mask_data]
dssp =  np.array(dssp)[mask_data][::200] # subsample
print(dssp.shape)

# load ensemble and subsample
# load trajectory into MDAnalysis
u = mda.Universe(path + "../processed.pdb",
                 path + f"../dir{sim}/traj{sim}.xtc")

# subsample
frames = np.arange(len(u.trajectory))[::200][:2501]
print(len(frames))

# load helical profile
frachelix = pd.read_csv(data_path+'300K_noHMR_perres_helicity_300K.csv')
residues = frachelix['Residue']
helicity = frachelix['Fraction helix']

# --- Compute Hfrac for each frame ---
Hfrac = np.array([helical_fraction_H(frame) for frame in dssp])

# --- Select top N frames ---
top_idx = np.argsort(Hfrac)[::-1][:n_structures]
print("Top indices:", top_idx)
print("Top Hfrac values:", Hfrac[top_idx])

# Build per-atom B-factors from per-residue helicity
bfactors = np.zeros(u.atoms.n_atoms)

for i, res in enumerate(u.residues):
    h = float(helicity.values[i])     
    bfactors[res.atoms.indices] = h

# Write selected frames
for rank, idx in enumerate(top_idx):
    traj_idx = frames[idx]
    u.trajectory[traj_idx]             # go to frame
    u.atoms.tempfactors = bfactors

    out_name = os.path.join(output_path, f"topH_{rank:02d}_frame{idx}.pdb")
    with PDBWriter(out_name) as W:
        W.write(u.atoms)

print("Done.")

(2501,)
2501
Top indices: [1028  143 1123  668  183  144  142 1939 1854  364]
Top Hfrac values: [0.82352941 0.82352941 0.82352941 0.76470588 0.76470588 0.76470588
 0.76470588 0.76470588 0.76470588 0.70588235]
Done.


/home/thd711/.conda/envs/HDXproject/lib/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:345: UserWarning: Unknown element  found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/home/thd711/.conda/envs/HDXproject/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: 
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))
/home/thd711/.conda/envs/HDXproject/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/thd711/.conda/envs/HDXproject/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1176: UserWarning: Found missing chainIDs. Corresponding atoms will use value of 'X'
  warnings.warn("Found mis

In [43]:
# ACTR
output_path = './pdbs_for_rendering/ACTR/'
path = '../OPES_multiT/ACTR/NEW_PRODUCTION_300K_2fs_80steps/alldata/'
data_path = './plots_ACTR/'
sim = 0
n_structures = 10

def helical_fraction_H(ss_line):
    helix_chars = {'H'}
    n_residues = len(ss_line)
    n_helical = sum(1 for c in ss_line if c in helix_chars)
    return n_helical / n_residues

data = read_colvar(path+f'COLVAR.{sim}')
with open(path+f'dssp{sim}.dat') as f:
    dssp = [line.strip() for line in f]
# Time arrays in μs
time_data = data[:, 0] / 1e6        # ps -> μs
# Apply cutoff at 5 µs
mask_data = time_data <= 5.0 
data = data[mask_data]
dssp =  np.array(dssp)[mask_data][::200] # subsample
print(dssp.shape)

# load ensemble and subsample
# load trajectory into MDAnalysis
u = mda.Universe(path + "../processed.pdb",
                 path + f"../dir{sim}/traj{sim}.xtc")

# subsample
frames = np.arange(len(u.trajectory))[::200][:2501]
print(len(frames))

# load helical profile
frachelix = pd.read_csv(data_path+'300K_perres_helicity_300K.csv')
residues = frachelix['Residue']
helicity = frachelix['Fraction helix']

# --- Compute Hfrac for each frame ---
Hfrac = np.array([helical_fraction_H(frame) for frame in dssp])

# --- Select top N frames ---
top_idx = np.argsort(Hfrac)[::-1][:n_structures]
print("Top indices:", top_idx)
print("Top Hfrac values:", Hfrac[top_idx])

# Build per-atom B-factors from per-residue helicity
bfactors = np.zeros(u.atoms.n_atoms)

for i, res in enumerate(u.residues):
    h = float(helicity.values[i])     
    bfactors[res.atoms.indices] = h

# Write selected frames
for rank, idx in enumerate(top_idx):
    traj_idx = frames[idx]
    u.trajectory[traj_idx]             # go to frame
    u.atoms.tempfactors = bfactors

    out_name = os.path.join(output_path, f"topH_{rank:02d}_frame{idx}.pdb")
    with PDBWriter(out_name) as W:
        W.write(u.atoms)

print("Done.")

(2501,)
2501
Top indices: [2235 2237 2248 2225 2232 2250 2234 2233 2253 2236]
Top Hfrac values: [0.53521127 0.52112676 0.47887324 0.47887324 0.45070423 0.45070423
 0.45070423 0.45070423 0.43661972 0.43661972]


/home/thd711/.conda/envs/HDXproject/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/thd711/.conda/envs/HDXproject/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1176: UserWarning: Found missing chainIDs. Corresponding atoms will use value of 'X'
  warnings.warn("Found missing chainIDs."


Done.


In [47]:
# fragment ACTR
output_path = './pdbs_for_rendering/fragment_ACTR/'
path = '../OPES_multiT/fragment_helix_ACTR/NEW_PRODUCTION_OPES_multiT_300K_noHMR/alldata/'
data_path = './plots_fragmentACTR/'
sim = 0
n_structures = 10

def helical_fraction_H(ss_line):
    helix_chars = {'H'}
    n_residues = len(ss_line)
    n_helical = sum(1 for c in ss_line if c in helix_chars)
    return n_helical / n_residues

data = read_colvar(path+f'COLVAR.{sim}')[::5]
with open(path+f'dssp{sim}.dat') as f:
    dssp = [line.strip() for line in f]
# Time arrays in μs
time_data = data[:, 0] / 1e6        # ps -> μs
# Apply cutoff at 5 µs
mask_data = time_data <= 5.0 
data = data[mask_data]
dssp =  np.array(dssp)[mask_data][::200] # subsample
print(dssp.shape)

# load ensemble and subsample
# load trajectory into MDAnalysis
u = mda.Universe(path + "../processed.pdb",
                 path + f"../dir{sim}/traj{sim}.xtc")

# subsample
frames = np.arange(len(u.trajectory))[::200][:2501]
print(len(frames))

# load helical profile
frachelix = pd.read_csv(data_path+'300K_perres_helicity_300K.csv')
residues = frachelix['Residue']
helicity = frachelix['Fraction helix']

# --- Compute Hfrac for each frame ---
Hfrac = np.array([helical_fraction_H(frame) for frame in dssp])

# --- Select top N frames ---
top_idx = np.argsort(Hfrac)[::-1][:n_structures]
print("Top indices:", top_idx)
print("Top Hfrac values:", Hfrac[top_idx])

# Build per-atom B-factors from per-residue helicity
bfactors = np.zeros(u.atoms.n_atoms)

for i, res in enumerate(u.residues):
    h = float(helicity.values[i])     
    bfactors[res.atoms.indices] = h

# Write selected frames
for rank, idx in enumerate(top_idx):
    traj_idx = frames[idx]
    u.trajectory[traj_idx]             # go to frame
    u.atoms.tempfactors = bfactors

    out_name = os.path.join(output_path, f"topH_{rank:02d}_frame{idx}.pdb")
    with PDBWriter(out_name) as W:
        W.write(u.atoms)

print("Done.")

(2501,)


/home/thd711/.conda/envs/HDXproject/lib/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:345: UserWarning: Unknown element  found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/home/thd711/.conda/envs/HDXproject/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: 
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


2501
Top indices: [ 603  573  609 1332  558  597  604  596  562  595]
Top Hfrac values: [0.51162791 0.51162791 0.51162791 0.51162791 0.51162791 0.51162791
 0.51162791 0.51162791 0.51162791 0.51162791]
Done.


/home/thd711/.conda/envs/HDXproject/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/thd711/.conda/envs/HDXproject/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1176: UserWarning: Found missing chainIDs. Corresponding atoms will use value of 'X'
  warnings.warn("Found missing chainIDs."


In [3]:
# HTT
output_path = './pdbs_for_rendering/HTTex1_16Q/'
path = '../OPES_multiT/HTTex1_16Q/NEW_PROD_OPES_multiT_100steps/alldata/'
data_path = './plots_HTTex1_16Q/'
sim = 3
n_structures = 10

def helical_fraction_H(ss_line):
    helix_chars = {'H'}
    n_residues = len(ss_line)
    n_helical = sum(1 for c in ss_line if c in helix_chars)
    return n_helical / n_residues

data = read_colvar(path+f'COLVAR.{sim}')
with open(path+f'dssp{sim}.dat') as f:
    dssp = [line.strip() for line in f]
# Time arrays in μs
time_data = data[:, 0] / 1e6        # ps -> μs
# Apply cutoff at 2 µs
mask_data = time_data <= 2.0 
data = data[mask_data]
dssp =  np.array(dssp)[mask_data][::100] # subsample
print(dssp.shape)

# load ensemble and subsample
# load trajectory into MDAnalysis
u = mda.Universe(path + "../processed.pdb",
                 path + f"../dir{sim}/traj{sim}.xtc")

# subsample
frames = np.arange(len(u.trajectory))[::100][:2001]
print(len(frames))

# load helical profile
frachelix = pd.read_csv(data_path+'300K_perres_helicity_293K.csv')
residues = frachelix['Residue']
helicity = frachelix['Fraction helix']

# --- Compute Hfrac for each frame ---
Hfrac = np.array([helical_fraction_H(frame) for frame in dssp])

# --- Select top N frames ---
top_idx = np.argsort(Hfrac)[::-1][:n_structures]
print("Top indices:", top_idx)
print("Top Hfrac values:", Hfrac[top_idx])

# Build per-atom B-factors from per-residue helicity
bfactors = np.zeros(u.atoms.n_atoms)

for i, res in enumerate(u.residues):
    h = float(helicity.values[i])     
    bfactors[res.atoms.indices] = h

# Write selected frames
for rank, idx in enumerate(top_idx):
    traj_idx = frames[idx]
    u.trajectory[traj_idx]             # go to frame
    u.atoms.tempfactors = bfactors

    out_name = os.path.join(output_path, f"topH_{rank:02d}_frame{idx}.pdb")
    with PDBWriter(out_name) as W:
        W.write(u.atoms)

print("Done.")

(2001,)


/home/thd711/.conda/envs/HDXproject/lib/python3.8/site-packages/MDAnalysis/topology/PDBParser.py:345: UserWarning: Unknown element  found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/home/thd711/.conda/envs/HDXproject/lib/python3.8/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: 
  warnings.warn("Failed to guess the mass for the following atom types: {}".format(atom_type))


2001
Top indices: [1205 1202 1201 1213 1212 1200 1197 1196 1206 1209]
Top Hfrac values: [0.39393939 0.39393939 0.37878788 0.36363636 0.36363636 0.36363636
 0.36363636 0.36363636 0.34848485 0.34848485]


/home/thd711/.conda/envs/HDXproject/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/thd711/.conda/envs/HDXproject/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1176: UserWarning: Found missing chainIDs. Corresponding atoms will use value of 'X'
  warnings.warn("Found missing chainIDs."


Done.
